In [38]:
%cd /Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt

/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt


In [39]:
path = '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/'

CATEGORIES = ['MGMT_negative', 'MGMT_positive']

block_h, block_w = (32, 32)
# changed the stride
stride = 3

Case_Num = 5

In [40]:
# Function Definition --> Backup Original Data For Safety Purpose

def Backup():
    import shutil
    print('Creating Backup')
    # Copy data for backup
    try:
        shutil.copytree(PATH, Org_Dir)
    except Exception as e:
        print(e)

    # Copy data for working
    try:
        shutil.copytree(Org_Dir, Work_Dir)
    except Exception as e:
        print(e)

    # Deleting folder, listed in the CATEGORIES list, after creating Cases
    for cate in CATEGORIES:
        try:
            shutil.rmtree(PATH + cate)  # Deleting Folders of CATEGORIES list
        except Exception as e:
            print(e)
    print('Backup Created')

In [41]:
# Function Definition --> Generate Tumor Images of Patients
import os
def Generate_images():
    from tqdm import tqdm
    print('Generating Images')
    try:
        workdir = os.listdir(Work_Dir)
        if '.DS_Store' in workdir:
            workdir.remove('.DS_Store')
        for case in workdir:
            Case_Path = os.path.join(Work_Dir, case + '/')
            casepath = os.listdir(Case_Path)
            if '.DS_Store' in casepath:
                casepath.remove('.DS_Store')
            for split in casepath:
                Split_Path = os.path.join(Case_Path, split + '/')
                splitpath = os.listdir(Split_Path)
                if '.DS_Store' in splitpath:
                    splitpath.remove('.DS_Store')
                for category in tqdm(splitpath):
                    Category_Path = os.path.join(Split_Path, category + '/')
                    os.chdir(Category_Path)
                    list_of_patients = []
                    categorypath = os.listdir(Category_Path)
                    if '.DS_Store' in categorypath:
                        categorypath.remove('.DS_Store')
                    for file in tqdm(categorypath):
                        patient = file.split('_')[0] + '_' + file.split('_')[1]
                        if patient not in list_of_patients:
                            list_of_patients.append(patient)
                            flair = nib.load('{}_t1ce.nii.gz'.format(patient))
                            flair_data = flair.get_fdata() #Converting nii to 3d np array
                            seg_mask = nib.load('{}_seg.nii.gz'.format(patient))
                            seg_mask_data = seg_mask.get_fdata() #Converting nii to 3d np array

                            #Extracting only those layers from mask hich have non zero values
                            z = np.any(seg_mask_data, axis=(0, 1))
                            zmin, zmax = np.where(z)[0][[0, -1]] #zmin & zmax saves the corresponding layer numbers of tumor regions

                            #Creating a new mask to remove segmentation
                            d = seg_mask_data
                            for layer in range(zmin,zmax+1):
                                nonzero = np.nonzero(d[:,:,layer])
                                r = nonzero[0]
                                c = nonzero[1]
                                if (r.size == 0 or c.size == 0):
                                    continue
                                rmin = np.min(r)
                                rmax = np.max(r)
                                cmin = np.min(c)
                                cmax = np.max(c)
                                d[rmin:rmax+1, cmin:cmax+1, layer] = 1 #Replacing tumor region values by 1

                            #Multiplying flair data with new mask
                            tumor = np.multiply(flair_data,d)

                            #Removing zero valued layers
                            tumor_layers = tumor[:,:,~(tumor==0).all((0,1))]

                            #converting to jpeg files
                            Cropped_list = [] #list containing cropped 2d layers of tumor region
                            for lay in range(0,tumor_layers.shape[2]):
                                coords = np.argwhere(tumor_layers[:,:,lay])
                                x_min, y_min = coords.min(axis=0)
                                x_max, y_max = coords.max(axis=0)
                                cropped = tumor_layers[x_min:x_max+1, y_min:y_max+1, lay]
                                cropped *= (255.0/cropped.max()) #normalization/scaling
                                Cropped_list.append(cropped)

                            frame = 0
                            for item in Cropped_list:
                                if((item.shape[0]*item.shape[1]) >= 300):
                                    frame = frame + 1
                                    im = Image.fromarray(item)
                                    im = im.convert('L')
                                    width, height = im.size
                                    if(height < 32 and width >= 32):
                                        im = im.resize((32, width))
                                    elif(height >= 32 and width < 32):
                                        im = im.resize((height, 32))
                                    elif(height < 32 and width < 32):
                                        im = im.resize((32,32))
                                    im.save("{}_img_{}.png".format(patient,frame))
                                    im.close()

                            #Removing unwanted nii files
                            niipath = os.listdir(os.getcwd())
                            if '.DS_Store' in niipath:
                                niipath.remove('.DS_Store')
                            for item in niipath:
                                try:
                                    if item.startswith(patient) and item.endswith(".gz"):
                                        os.remove(item)
                                except Exception as e:
                                    print(e)
    except Exception as e:
        print(e)
    os.chdir(Work_Dir)
    print('Images Generated')

In [42]:
# Function Definition --> Dividing Datasets into number of cases for Cross Validation

def Creating_Cases(Case_Num):
    import shutil
    import os
    print('Creating Cases')
    for case in range(Case_Num):
        CASES.append('Case_'+ str(case + 1))

    # Creating Case-1
    Case = Work_Dir + 'Case_1'
    try:
        shutil.copytree(Work_Dir, Case)
    except Exception as e:
        print(e)

    # Creating Remaining Cases
    try:
        for case in range(Case_Num - 1):
            shutil.copytree(Case, Work_Dir + 'Case_' + str(case+2))
    except Exception as e:
        print(e)

    # Deleting folder, listed in the CATEGORIES list, after creating Cases
    for cate in CATEGORIES:
        try:
            shutil.rmtree(Work_Dir + cate)  # Deleting Folders of CATEGORIES list
        except Exception as e:
            print(e)
    print('Cases Created')

In [43]:
# Function Definition --> Seprating_Test_Data to perform K-fold Cross Validation

def Seprating_Test_Data():
    import splitfolders
    import shutil
    from tqdm import tqdm
    print('Seprating Test Data')
    i = 0
    workdir=os.listdir(Work_Dir)
    if '.DS_Store' in workdir:
        workdir.remove('.DS_Store')
        print("Removed .DS_Store")
    for case in tqdm(workdir):
        try:
            Case_Path = os.path.join(Work_Dir, case + '/')
            casepath = os.listdir(Case_Path)
            if '.DS_Store' in casepath:
                casepath.remove('.DS_Store')
            for Type in casepath:
                Type_Path = os.path.join(Case_Path, Type + '/')
                typepath = os.listdir(Type_Path)
                if '.DS_Store' in typepath:
                    typepath.remove('.DS_Store')
                for patient in typepath:
                    Patient_Path = os.path.join(Type_Path, patient + '/')
                    patientpath = os.listdir(Patient_Path)
                    if '.DS_Store' in patientpath:
                        patientpath.remove('.DS_Store')
                    for file in patientpath:
                        file_Path = os.path.join(Patient_Path, file)
                        source = file_Path
                        destination = Type_Path
                        shutil.copy(source, destination)
                    shutil.rmtree(Type_Path + patient)
        except Exception as e:
            print(e)
        input_folder = Case_Path
        output_folder = input_folder
        i = i+1
        splitfolders.ratio(input_folder, output = output_folder, seed = i, ratio = (.8, .2), group_prefix =5)

        # Deleting folder, listed in the CATEGORIES list, after Seprating the data
        for cate in CATEGORIES:
            try:
                shutil.rmtree(Case_Path + cate)
            except Exception as e:
                print(e)

        # Rename folder 'val' to 'test'
        try:
            os.rename(Case_Path + 'val', Case_Path + 'test')
        except Exception as e:
            print(e)
    print('Test Data Seprated')

In [44]:
# Function Definition --> Create Image Blocks

def Creating_Image_Blocks(block_h, block_w, stride):
    import glob
    from PIL import Image
    from tqdm import tqdm
    print('Creating Image Blocks')
    workdir = os.listdir(Work_Dir)
    if '.DS_Store' in workdir:
        workdir.remove('.DS_Store')
    for case in workdir:
        Case_Path = os.path.join(Work_Dir, case + '/')
        casepath = os.listdir(Case_Path)
        if '.DS_Store' in casepath:
           casepath.remove('.DS_Store')

        for split in casepath:
            Split_Path = os.path.join(Case_Path, split + '/')
            splitpath = os.listdir(Split_Path)
            if '.DS_Store' in splitpath:
                splitpath.remove('.DS_Store')
            for category in tqdm(splitpath):
                Category_Path = os.path.join(Split_Path, category + '/')

                os.chdir(Category_Path)
                for img_file in glob.glob('*.png'):
                    
                    img = Image.open(img_file)
                    img_w, img_h = img.size

                    File_Name, extension = os.path.splitext(img_file)

                    try:
                        os.mkdir(File_Name)
                    except Exception as e:
                        print(e)

                    SaveDir = Category_Path + File_Name

                    frame_num = 0
                    count_row = 0
                    
                    for row in range(0, img_h, stride):
                        if (img_h - row >= block_h):
                            count_row += 1
                            count_column = 0

                            for col in range(0, img_w, stride):
                                if (img_w - col >= block_w):
                                    count_column += 1
                                    crop = img.crop((col, row, col + block_w, row + block_h))
                                    save_to= os.path.join(SaveDir + '/' + f'{File_Name}_{count_row}_{count_column}.png')
                                    crop.save(save_to.format(frame_num + 1))
                    
                    img.close()
                    # Deleting Image Files
                    os.remove(img_file)
    print('Image Blocks Created')

In [45]:
# All Function Calls
import os
import numpy as np
import nibabel as nib
from PIL import Image

PATH = path + 'Data/BRATS/mod_data/'
Org_Dir = PATH + 'Original_Data_Backup/'
Work_Dir = PATH + 'Working_Data/'

# Function Call --> Backup Original Data For Safety Purpose
# Backup()

# Function Call --> Dividing Datasets into number of cases for Cross Validation
CASES = []
Creating_Cases(Case_Num)

# Function Call --> Seprating_Test_Data to perform K-fold Cross Validation
Seprating_Test_Data()

# Function Call --> Generate Tumor Images of Patients
Generate_images()

# Function Call --> Create Image Blocks
Creating_Image_Blocks(block_h, block_w, stride)

os.chdir(path)
print('All Done')

Creating Cases
Cases Created
Seprating Test Data
Removed .DS_Store


Copying files: 400 files [00:08, 44.62 files/s]
Copying files: 400 files [00:09, 43.89 files/s]
Copying files: 400 files [00:06, 61.55 files/s]
Copying files: 400 files [00:07, 53.26 files/s]
Copying files: 400 files [00:09, 44.17 files/s]
100%|██████████| 5/5 [01:18<00:00, 15.79s/it]


Test Data Seprated
Generating Images


100%|██████████| 2/2 [00:49<00:00, 24.85s/it]


Images Generated
Creating Image Blocks


100%|██████████| 2/2 [03:57<00:00, 118.77s/it]

Image Blocks Created
All Done


In [46]:
# reconstucting working directory
def reconstruct():
    import shutil
    print('Reconstructing')
    # Deleting working directory
    shutil.rmtree(PATH + "Working_data/")
    # Copying data from backup
    try:
        shutil.copytree(Org_Dir, Work_Dir)
    except:
        print('Working directory already exists')

    print('Reconstruction complete')


In [47]:
# reconstruct()

In [48]:
# removing .DS_Store files
def remove_DS_Store():
    import os
    os.chdir(Work_Dir)
    workdir = os.listdir(Work_Dir)
    if '.DS_Store' in workdir:
        workdir.remove('.DS_Store')
        os.remove('.DS_Store')
        print("Removed .DS_Store from Working Directory list")
    for case in workdir:
        Case_Path = os.path.join(Work_Dir, case + '/')
        casepath = os.listdir(Case_Path)
        os.chdir(Case_Path)
        if '.DS_Store' in casepath:
            casepath.remove('.DS_Store')
            os.remove('.DS_Store')
            print("Removed .DS_Store from Case list")
        for split in casepath:
            Split_Path = os.path.join(Case_Path, split + '/')
            splitpath = os.listdir(Split_Path)
            os.chdir(Split_Path)
            if '.DS_Store' in splitpath:
                splitpath.remove('.DS_Store')
                os.remove('.DS_Store')
                print("Removed .DS_Store from Split list")
            for category in splitpath:
                Category_Path = os.path.join(Split_Path, category + '/')
                categorypath = os.listdir(Category_Path)
                os.chdir(Category_Path)
                if '.DS_Store' in categorypath:
                    os.remove('.DS_Store')
                    categorypath.remove('.DS_Store')
                    print("Removed .DS_Store from Category list")
                for folder in categorypath:
                    Folder_Path = os.path.join(Category_Path, folder + '/')
                    folderpath = os.listdir(Folder_Path)
                    os.chdir(Folder_Path)
                    if '.DS_Store' in folderpath:
                        os.remove('.DS_Store')
                        folderpath.remove('.DS_Store')
                        print("Removed .DS_Store from Folder list")
    
    

In [49]:
remove_DS_Store()

Removed .DS_Store from Working Directory list
Removed .DS_Store from Case list
Removed .DS_Store from Case list
Removed .DS_Store from Case list
Removed .DS_Store from Case list
Removed .DS_Store from Case list
